In [1]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Using device: cuda

GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import wandb

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader

from torch.utils import data


import copy
import os
import random
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import functools
from tqdm import tqdm
from datetime import datetime
import numpy as np
from core.datasets.vqa_motion_dataset import VQMotionDataset,DATALoader,VQVarLenMotionDataset,MotionCollator,VQFullMotionDataset
from einops import rearrange, reduce, pack, unpack
import librosa

## Our method

In [4]:
from utils.motion_process import recover_from_ric
import visualize.plot_3d_global as plot_3d
from glob import glob
def to_xyz(motion, mean ,std , j = 22):
    motion_xyz = recover_from_ric(motion.cpu().float()*std+mean, j)
    motion_xyz = motion_xyz.reshape(motion.shape[0],-1, j, 3)
    return motion_xyz

            
def sample_render(motion_xyz , name , save_path):
    print(f"render start")
    
    gt_pose_vis = plot_3d.draw_to_batch(motion_xyz.numpy(),None, [os.path.join(save_path,name + ".gif")])



In [5]:
from configs.config import cfg, get_cfg_defaults
from core.models.vqvae import VQMotionModel
from core.models.motion_regressor import MotionRegressorModel

cfg_vq = get_cfg_defaults()
cfg_vq.merge_from_file("/srv/scratch/sanisetty3/music_motion/motion_vqvae/configs/var_len_768_768_aist_vq.yaml")


cfg_trans = get_cfg_defaults()
cfg_trans.merge_from_file("/srv/scratch/sanisetty3/music_motion/motion_vqvae/configs/var_len_768_768_aist.yaml")



In [6]:
vqvae_model = VQMotionModel(cfg_vq.vqvae).eval()
pkg = torch.load(f"/srv/scratch/sanisetty3/music_motion/motion_vqvae/checkpoints/var_len/vq_768_768_mix/vqvae_motion_best_fid.pt", map_location = 'cpu')
print(pkg["steps"])
vqvae_model.load_state_dict(pkg["model"])
vqvae_model =vqvae_model.cuda()


tensor([295000.])


In [7]:
trans_model = MotionRegressorModel(args = cfg_trans.motion_trans , ignore_index=1025 ,pad_value=1025 ).eval()
pkg_trans = torch.load(f"/srv/scratch/sanisetty3/music_motion/motion_vqvae/checkpoints/generator/var_len/trans_768_768_albi_aist/trans_motion_best_fid.pt", map_location = 'cpu')
print(pkg_trans["steps"])
trans_model.load_state_dict(pkg_trans["model"])
trans_model =trans_model.cuda()


tensor([195000.])


In [90]:
os.makedirs("/srv/scratch/sanisetty3/music_motion/motion_vqvae/evals/generator/var_len/trans_768_768_albi_aist/", exist_ok=True)

## Motion generation

In [8]:
from core.datasets.vqa_motion_dataset import MotionCollatorConditional,VQVarLenMotionDatasetConditional, TransMotionDatasetConditional,TransMotionDatasetConditionalFull
from utils.eval_music import evaluate_music_motion_vqvae, evaluate_music_motion_trans,get_target_indices


In [9]:
def process_gen_output(gen_motion_indices , seq_len):
    eos_index = (gen_motion_indices == 1026).nonzero().flatten().tolist()
    # print(eos_index)
    pad_index = (gen_motion_indices == 1025).nonzero().flatten().tolist()
    # print(pad_index)
    bos_index = (gen_motion_indices == 1024).nonzero().flatten().tolist()
    # print(bos_index)
    stop_index = min([*eos_index , *pad_index , *bos_index, seq_len])

    gen_motion_indices_ = gen_motion_indices[:int(stop_index)]

    return gen_motion_indices_


In [10]:
    def get_target_indices2(batch , trans_model , sample_max = False , bos = 1024,pad = 1025,eos = 1026):
    ##batch size = 1

        inp, target = batch["motion"][:, :-1], batch["motion"][:, 1:]
        # print(target.shape)

        ##inp: b seqlen-1 target: b seqlen-1
        length = int(batch["motion_lengths"][0])
        # print(length)

        logits = trans_model(motion = inp.cuda() , mask = batch["motion_mask"][:,:-1].cuda()  , \
            context = batch["condition"].cuda(), context_mask = batch["condition_mask"].cuda())

        probs = torch.softmax(logits[0][:length], dim=-1)
        if sample_max:
            _, cls_pred_index = torch.max(probs, dim=-1)

        else:
            dist = torch.distributions.Categorical(probs)
            cls_pred_index = dist.sample()
        # print(cls_pred_index.shape)


        ## cls_pred_index - list

        eos_index = (cls_pred_index == eos).nonzero().flatten().tolist()
        # print(eos_index)
        pad_index = (cls_pred_index == pad).nonzero().flatten().tolist()
        # print(pad_index)
        bos_index = (cls_pred_index == bos).nonzero().flatten().tolist()
        # print(bos_index)
        stop_index = min([*eos_index , *pad_index , *bos_index, length-1])

        gen_motion_indices_ = cls_pred_index[:int(stop_index)]
        gt_motion_indices_ = target[target<bos]

        # print(gen_motion_indices_.dtype , gt_motion_indices_.dtype)


        gen_motion_indices_ = (gen_motion_indices_).contiguous().view(1,-1)
        gt_motion_indices_ = gt_motion_indices_.contiguous().view(1,-1)
        # print(gen_motion_indices_.shape,gt_motion_indices_.shape)


        return gen_motion_indices_ , gt_motion_indices_

In [11]:
train_ds = TransMotionDatasetConditionalFull("aist", split = "test",data_root = "/srv/scratch/sanisetty3/music_motion/AIST" , datafolder="joint_indices_max_400", window_size = -1)


100%|██████████| 40/40 [00:00<00:00, 894.99it/s]

Total number of motions 40


In [12]:
collate_fn2 = MotionCollatorConditional(dataset_name = "aist" , bos = 1024, pad = 1025, eos = 1026)


In [13]:
dl = DATALoader(train_ds , batch_size = 1,collate_fn=collate_fn2)


In [14]:
for reg_batch in dl:
    break
for k,v in reg_batch.items():
    print(k , v.shape)
    try:
        v = v.cuda()
    except:
        continue

motion torch.Size([1, 175])
motion_lengths torch.Size([1])
motion_mask torch.Size([1, 175])
names (1,)
condition torch.Size([1, 174, 128])
condition_mask torch.Size([1, 174])


In [17]:
music_name = str(reg_batch["names"]).split("_")[-2]
music_encoding=  np.load(os.path.join("/srv/scratch/sanisetty3/music_motion/AIST/music" , music_name + ".npy"))

In [15]:
inp, target = reg_batch["motion"][:, :-1], reg_batch["motion"][:, 1:]

In [43]:
inp[:,:1].shape

torch.Size([1, 1])

In [114]:
gen_motion_indices , gt_motion_indices = get_target_indices2(reg_batch , trans_model)

In [115]:
gen_motion_indices.shape

torch.Size([1, 140])

In [111]:
gen_motion_indices = trans_model.generate(start_tokens =torch.randint(0 , 1024 , (1,1)).cuda(),\
                                          seq_len=400 , \
                                          context = torch.Tensor(music_encoding)[None,...].cuda(), \
                                        context_mask=torch.ones((1 ,music_encoding.shape[0]) , dtype = torch.bool).cuda()
                                          )

100%|██████████| 400/400 [00:08<00:00, 45.13it/s]


In [112]:
gen_motion_indices = gen_motion_indices[gen_motion_indices<1024][None,...]

In [113]:
gen_motion_indices.shape

torch.Size([1, 145])

In [98]:
inp[:,:1]

torch.Size([1, 1])

In [103]:
torch.randint(0 , 1024 , (1,1))

tensor([[958]])

In [110]:
os.path.basename(musics[0])

'mWA0.npy'

## Generate 20s and save gen + gt

In [121]:
gt_path = "/srv/scratch/sanisetty3/music_motion/AIST/joint_indices_max_400/*"

In [122]:
gts = glob(gt_path)

In [125]:
gts[0]

'/srv/scratch/sanisetty3/music_motion/AIST/joint_indices_max_400/gBR_sFM_cAll_d06_mBR4_ch20.npy'

In [128]:
os.path.basename(gts[0]).split(".")[0].split('_')[-2]

'mBR4'

In [149]:
from itertools import groupby

a = gts

def get_domain(path):
    # Example: 'www.google.com' -> 'google.com'
    return '.'.join(os.path.basename(path).split(".")[0].split('_')[-2])

def choose_item(iterable):
    # Ex. input: ['www.google.com', 'google.com',  'www.google.com']
    # Ex. output: 'www.google.com' (longest string)
    return sorted(iterable, key=lambda x: -len(x))[0]

results = []
for domain,grp in groupby(sorted(a, key=get_domain), key=get_domain):
    results.append(choose_item(grp))


In [152]:
gt_motion_indices_path = results
len(gt_motion_indices_path)

60

In [136]:
def get_motion(music):
    for i in gt_motion_indices_path:
        if os.path.basename(i).split(".")[0].split('_')[-2] == music:
            return i

In [147]:
save_pth = "/srv/scratch/sanisetty3/music_motion/motion_vqvae/evals/generator/var_len/trans_768_768_albi_aist/"

In [154]:
musics = glob("/srv/scratch/sanisetty3/music_motion/AIST/music/*")
for music_path in musics:
    

    # music_name = str(reg_batch["names"]).split("_")[-2]
    music_encoding=  np.load(os.path.join(music_path))
    music_name = os.path.basename(music_path).split(".")[0]
    print(music_name)
    motion_indices_path = get_motion(music_name)
    gt_motion_indices = torch.LongTensor(np.load(motion_indices_path))[None,...]
    gen_motion_indices = inp[:,:1]
    while gen_motion_indices.shape[1]<=400:
        gen_motion_indices = trans_model.generate(start_tokens =gen_motion_indices.cuda(),\
                                              seq_len=400 , \
                                              context = torch.Tensor(music_encoding)[None,...].cuda(), \
                                            context_mask=torch.ones((1 ,music_encoding.shape[0]) , dtype = torch.bool).cuda()
                                              )
        gen_motion_indices = gen_motion_indices[gen_motion_indices<1024][None,...]
        print(gen_motion_indices.shape[1])
        
    
    quant , out_motion = vqvae_model.decode(gen_motion_indices.cuda())
    np.save(os.path.join(save_pth , music_name+".npy" ,) , out_motion.cpu().numpy())

    sample_render(to_xyz(out_motion[0:1].detach().cpu(),mean = train_ds.mean , std = train_ds.std), name = music_name+"" , save_path = save_pth)
    
    quant , out_motion = vqvae_model.decode(gt_motion_indices.cuda())
    sample_render(to_xyz(out_motion[0:1].detach().cpu(),mean = train_ds.mean , std = train_ds.std), name = music_name+"_gt" , save_path = save_pth)
    
    gt_motion_indices

mWA0


100%|██████████| 400/400 [00:08<00:00, 44.88it/s]


401
render start
render start
mHO0


100%|██████████| 400/400 [00:08<00:00, 44.96it/s]


401
render start
render start
mJB0


...

401
render start
render start
mJB3


100%|██████████| 400/400 [00:08<00:00, 44.87it/s]


401
render start
render start
mMH4


100%|██████████| 400/400 [00:08<00:00, 45.01it/s]


401
render start
render start


In [35]:
processed_gen_output = process_gen_output(gen_motion_indices[0] , 400)

In [36]:
processed_gen_output.shape

torch.Size([119])

In [114]:
quant , out_motion = vqvae_model.decode(gen_motion_indices.cuda())
sample_render(to_xyz(out_motion[0:1].detach().cpu(),mean = train_ds.mean , std = train_ds.std), "rnd_motion" , "/srv/scratch/sanisetty3/music_motion/motion_vqvae/evals/decode_test")

render start


## Genre specific FID

In [ ]:
Need to group motion 